In [ ]:
import os
import sys
import math
import logging
import structlog
from pathlib import Path
import json

import tomli
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

In [ ]:
logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

In [ ]:
import pytanis
from pytanis import GSheetClient, PretalxClient, HelpDeskClient
from pytanis.review import Col
from pytanis.helpdesk import Mail, Recipient, MailClient

In [ ]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

In [ ]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)

# Get all the Reviewers

In [ ]:
gsheet_client = GSheetClient()
gsheet_df = gsheet_client.gsheet_as_df(cfg['reviewer_spread_id'], cfg['reviewer_work_name'])
# rename columns to stick to our convention
col_map = {
 "Topics you want to review": Col.track_prefs,
 "Email address": Col.email,
 "Name": Col.speaker_name,
 "Affiliation": Col.affiliation,
 "Who do you know from the Committee?": Col.committee_contact,
 "Availability during the Review Period": Col.availability,
 "Additional comments regarding your availability during the review period.": Col.availability_comment,
 "Activated in Pretalx": Col.pretalx_activated,
 "Do you want your name to be listed as a reviewer on the conference website?": Col.public,
 "Wants all proposals": Col.all_proposals,
 "Any additional comments for the Program Committee": Col.comment,
 "Committee Member": Col.committee_member
}
gsheet_df.rename(columns=col_map, inplace=True)

In [ ]:
# add column to address people nicely
gsheet_df[Col.address_as] = gsheet_df[Col.speaker_name].apply(lambda x: x.split()[0].title())

In [ ]:
reviewers_all = gsheet_df[[Col.speaker_name, Col.email, Col.address_as]]
reviewers_all = reviewers_all.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x[Col.email], address_as=x[Col.address_as]), axis=1).to_list()

In [ ]:
# determine reviewers having not even activated the Pretalx Acccount as well as non Committee members and mark them as recipients
reviewers_not_activated = gsheet_df.loc[gsheet_df[Col.pretalx_activated].isna() & gsheet_df[Col.committee_member].isna(), [Col.speaker_name, Col.email, Col.address_as]]
reviewers_not_activated = reviewers_not_activated.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x[Col.email], address_as=x[Col.address_as]), axis=1).to_list()

In [ ]:
# for activated reviewers we take the e-mail address of their pretalx account
reviewers_activated = gsheet_df.loc[gsheet_df[Col.pretalx_activated].notnull() & gsheet_df[Col.committee_member].isna(), [Col.speaker_name, 'Pretalx Mail', Col.address_as]]
reviewers_activated = reviewers_activated.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x['Pretalx Mail'], address_as=x[Col.address_as]), axis=1).to_list()

# Initial Mail to Reviewers for Onboarding

In [ ]:
mail_body = """
Hi {recipient.address_as}!

I hope this message finds you well. As the Chair of the Programme Committee for PyConDE & PyData,
it's my pleasure to welcome you to our team. Your contribution is vital to the success of our upcoming event.

Today, you should have received an invitation from noreply@pretalx.com to join Pretalx, our platform for
managing conference submissions. Please follow the link in the email to activate your account and
ensure you keep your login credentials safe.

We will be sending out more detailed information and guidelines in the new year.
In the meantime, if you encounter any issues signing up or have any questions,
feel free to reach out to us at program24@pycon.de.

Until then, I wish you a joyful Christmas season and a fantastic start to the New Year!

Warm regards,

Florian Wilhelm
Program Committee Chair
PyCon DE & PyData Berlin 2024
"""

In [ ]:
mail = Mail(
    subject="Welcome to PyConDE & PyData 2024 Review Team - Important Account Activation Information",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=reviewers_not_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail, dry_run=True)
assert not errors

# Mail to Reviewers that haven't activated their account in Pretalx

In [ ]:
mail_body = """
Hi {recipient.address_as} and welcome to the PyConDE / PyData 2024 review team!

I hope this message finds you well. As the Chair of the Programme Committee for PyConDE & PyData,
it's my pleasure to welcome you to our team. Your contribution is vital to the success of our upcoming event.

In the last few days, you should have received an invitation from noreply@pretalx.com to join Pretalx, our platform for
managing conference submissions. Have you checked your SPAM folder yet as it seems you haven't activated your
Pretalx Account for reviewing. Please follow the link in the Pretalx email to activate your account and ensure you keep
your login credentials safe. PLEASE DO THIS NOW :-)

We will be sending out more detailed information and guidelines in the new year.
In the meantime, if you encounter any issues signing up, can't find the Pretalx email or have any questions,
feel free to reach out to us at program24@pycon.de.

All the best,
Program Committee
PyCon DE & PyData Berlin 2024
"""

In [ ]:
mail = Mail(
    subject="Important! Please activate your Pretalx Account for PyConDE & PyData 2024",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=reviewers_not_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail, dry_run=True)
assert not errors

## Mail regarding the Review Onboarding

In [ ]:
mail_body = """
Hi {recipient.address_as},

the end of the year 2023 is near and we are all excited about what's to come in 2024.
The Programme Committee is doing right now the final preparations to organise the review process.

We have just updated our Reviewer Guidelines (link below) for you to read before the actual review
process will start on Monday, 9th of January and the deadline is Wednesday, 31st of January Midnight.
To meet your fellow reviewers, have a nice chat, talk to the programme committee and ask questions
personally, we also offer two non-mandatory Get Togethers on the 4th & 11th of January (details below).

If you have any questions, need help, don't hesitate contacting us at program24@pycon.de.
We appreciate your help very much and want to make sure that you are having a good time.

Thank you very much {recipient.address_as} for your support!

Summary:
* Review period: 9. January 2024 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2024/reviews/
* Reviewer Guidelines (http://bit.ly/pyconde24-reviewer-guidelines)
* [Nonobligatory] PyCon 2024 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday, 4. January 2024 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/zsn-avdq-yhy
  - Thursday, 11. January 2024 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/zsn-avdq-yhy
* Contact program24@pycon.de for support if needed

IMPORTANT: If you haven't signed up on Pretalx yet, please search for a mail from `noreply@pretalx.com`
           in your mailbox, also SPAM folder and confirm it. It's necessary for our assignment of proposals for review.


All the best,
Program Committee
PyCon DE & PyData Berlin 2024
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2024] Information about the Review Process in 2024",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=reviewers_all
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail, dry_run=True)
assert not errors

## Mail regarding the Review Onboarding 2 only to activated reviewers

In [ ]:
mail_body = """
Hi {recipient.address_as},

we hope you had a great holiday season, after all you were nice and activated your Pretalx account,
so Santa must have been generous with you ;-) One last bit of information about the review process for
this year. More to come in 2024...

We just assigned every reviewer initially 10 proposals so that you have a chance to familiarize yourself with
Pretalx before the Reviewer's Get Together dates (see below) to ask questions. It's *not* mandatory to participate
and the official review phase will start on January 9th in 2024. So this is just a tidbit of what's to come in 2024.

See you next year and thank you very much {recipient.address_as} for your support!

Summary:
* Review period: 9. January 2024 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2024/reviews/
* Reviewer Guidelines (http://bit.ly/pyconde24-reviewer-guidelines)
* [Nonobligatory] PyCon 2024 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday, 4. January 2024 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/zsn-avdq-yhy
  - Thursday, 11. January 2024 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/zsn-avdq-yhy
* Contact program24@pycon.de for support if needed

All the best,
Program Committee
PyCon DE & PyData Berlin 2024
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2024] One more thing about the Review Process in 2024",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=reviewers_activated[53:]
)

In [ ]:
reviewers_activated[53:]

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail, dry_run=False)
assert not errors

# Mail to Reviewers activated in Pretalx

## Mail regarding the Review Onboarding

In [ ]:
mail_body = """
Hi {recipient.address_as},

the end of the year 2023 is near and we are all excited about what's to come in 2024.
The Programme Committee is doing right now the final preparations to organise the review process.

We have just updated our Reviewer Guidelines (link below) for you to read before the actual review
process will start on Monday, 9th of January and the deadline is Wednesday, 31st of January Midnight.
To meet your fellow reviewers, have a nice chat, talk to the programme committee and ask questions
personally, we also offer two non-mandatory Get Togethers on the 4th & 11th of January (details below).

If you have any questions, need help, don't hesitate contacting us at program24@pycon.de.
We appreciate your help very much and want to make sure that you are having a good time.

Thank you very much {recipient.address_as} for your support!

Summary:
* Review period: 9. January 2024 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2024/reviews/
* Reviewer Guidelines (http://bit.ly/pyconde24-reviewer-guidelines)
* [Nonobligatory] PyCon 2024 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday, 4. January 2024 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/zsn-avdq-yhy
  - Thursday, 11. January 2024 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/zsn-avdq-yhy
* Contact program24@pycon.de for support if needed

IMPORTANT: If you haven't signed up on Pretalx yet, please search for a mail from `noreply@pretalx.com`
           in your mailbox, also SPAM folder and confirm it. It's necessary for our assignment of proposals for review.


All the best,
Program Committee
PyCon DE & PyData Berlin 2024
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2024] Information about the Review Process in 2024",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=reviewers
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail, dry_run=True)
assert not errors

In [ ]:
errors

In [ ]:
activated_reviewers = gsheet_df.loc[~gsheet_df[Col.pretalx_activated].isna()]

### Analyse the current reviews and determine top X% reviewers

In [ ]:
pretalx_client = PretalxClient()
n_reviews, reviews = pretalx_client.reviews(cfg['event_name'])
reviews = list(reviews)

In [ ]:
scored_reviews_df = pd.DataFrame([{"user": r.user, "score": r.score, "n_reviews": r.submission} for r in reviews if r.score is not None])
scored_reviews_df = scored_reviews_df.groupby("user").count()[["n_reviews"]]
scored_reviews_df['top_perc'] =  (1. - scored_reviews_df.rank(pct=True)["n_reviews"])

def top_perc_text(perc):
    if perc <= 0.1:
        return "top 10%"
    elif perc <= 0.25: 
        return "top 25%"
    elif perc <= 0.5: 
        return "top 50%"
    elif perc <= 0.75: 
        return "top 75%"
    elif perc <= 0.90: 
        return "top 90%"
    else:
        return "top 100% ;-)"

scored_reviews_df['top_perc_text'] = scored_reviews_df['top_perc'].apply(top_perc_text)

### Merge back with all reviewers and generate a nice feedback message

In [ ]:
activated_reviewers = pd.merge(activated_reviewers, scored_reviews_df, right_on='user', left_on='Pretalx Name', how='left')
activated_reviewers["n_reviews"].fillna(0., inplace=True)

In [ ]:
def get_feedback(x):
    if x['n_reviews'] == 0.:
        return "So far you haven't reviewed any proposals, it's time to get started!"
    else:
        return f"Thanks for having already started to reviews! You are a champion and currently in the {x['top_perc_text']} of all reviewers!"

activated_reviewers["feedback"] = activated_reviewers.apply(get_feedback, axis=1)

### Create and send an individual e-mail to each active reviewer

In [ ]:
active_recipients = activated_reviewers.apply(lambda x: Recipient(name=x[Col.speaker_name], 
                                                                  email=x[Col.email], 
                                                                  address_as=x[Col.address_as], 
                                                                  data={"feedback": x["feedback"]}),  axis=1).to_list()

In [ ]:
mail_body = """
Hi {recipient.address_as}!

The review of proposals for the PyConDE / PyData is already in full swing and we are happy to have you on board!
{recipient.data.feedback}

Please check back frequently into Pretalx as we will soon assign proposals more dynamically. 
PyConDE / PyData is completely community driven by volunteers like you and we highly appreciate your support!
We will keep you updated.

Information from our past e-mails:
* Reviewer Guidelines (https://bit.ly/pyconde23-reviewer-guidelines)
* [Nonobligatory] 2nd live “Meet and Greet” session: 17 January, 17:00 (CET) in Gather Town (https://bit.ly/pyconde23-meet-reviewers)
* Review to be finished by: 31 January, 00:00 (CET)
* Contact program23@pycon.de for support if needed

Thank you very much {recipient.address_as} for your support!


All the best,
Program Committee
PyCon DE & PyData Berlin 2023
"""

In [ ]:
mail = Mail(
    subject="Update for you on the review process for PyCon DE / PyData!",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=active_recipients
)

In [ ]:
responses, errors = mail_client.send(mail, dry_run=DRY_RUN)
assert not errors

## Mail about first reviews and Meet & Greet Sessions

In [ ]:
mail_body = """
Hi {recipient.address_as}!

The review of proposals for the PyConDE / PyData is already in full swing and we are happy to have you on board!
{recipient.data.feedback}

Please check back frequently into Pretalx as we will soon assign proposals more dynamically. 
PyConDE / PyData is completely community driven by volunteers like you and we highly appreciate your support!
We will keep you updated.

Information from our past e-mails:
* Reviewer Guidelines (https://bit.ly/pyconde23-reviewer-guidelines)
* [Nonobligatory] 2nd live “Meet and Greet” session: 17 January, 17:00 (CET) in Gather Town (https://bit.ly/pyconde23-meet-reviewers)
* Review to be finished by: 31 January, 00:00 (CET)
* Contact program23@pycon.de for support if needed

Thank you very much {recipient.address_as} for your support!


All the best,
Program Committee
PyCon DE & PyData Berlin 2023
"""

In [ ]:
mail = Mail(
    subject="Update for you on the review process for PyCon DE / PyData!",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=active_recipients
)

In [ ]:
responses, errors = mail_client.send(mail, dry_run=DRY_RUN)
assert not errors

## Reminder e-mail to everyone activated

In [ ]:
def get_feedback(x):
    if x['n_reviews'] == 0.:
        return "So far you haven't reviewed any proposals, it seems. Now it's really time to get started :-)\nPlease let us know if you are not able to review for some reason. In this case, we must assign your proposals to others soon."
    else:
        return f"Thanks that you already supported us so much! We are close to the finish line. Go for it, champion!"

activated_reviewers["feedback"] = activated_reviewers.apply(get_feedback, axis=1)

In [ ]:
active_recipients = activated_reviewers.apply(lambda x: Recipient(name=x[Col.speaker_name], 
                                                                  email=x[Col.email], 
                                                                  address_as=x[Col.address_as], 
                                                                  data={"feedback": x["feedback"]}),  axis=1).to_list()

In [ ]:
mail_body = """
Hi {recipient.address_as}!

We hope you had a great start into the new week :-)
There are only 8 days left until the deadline of our review process on 31 January, 00:00 (CET). 
We have reached now 70% of all the reviews we need, that's awesome! But still a few miles to go for us.
{recipient.data.feedback}

Please check back frequently into Pretalx as there might be only a few more proposals in case reviewers dropped out.
PyConDE / PyData is completely community driven by volunteers like you and we highly appreciate your support!
We will keep you updated.

Information from our past e-mails:
* Reviewer Guidelines (https://bit.ly/pyconde23-reviewer-guidelines)
* Contact program23@pycon.de for support if needed

Thank you very much {recipient.address_as} for your support!


All the best,
Program Committee
PyCon DE & PyData Berlin 2023
"""

In [ ]:
mail = Mail(
    subject="Reminder: deadline of the review process for PyCon DE / PyData is coming closer!",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=active_recipients
)

In [ ]:
responses, errors = mail_client.send(mail, dry_run=DRY_RUN)
assert not errors